In [1]:
import pandas as pd
import pandas_datareader as pdr

import requests as rq
import datetime as dt

import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

C:\Users\Tanushree\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
#Reading the Cleaned excel file from previous Lab Session.
Lab3 = pd.read_excel(r"C:\Users\Tanushree\Desktop\UNIVERSITY\Quarter 4\Dashboard\Lab1_Session\Session1.xlsx",sheetname = 0)

#Getting the first 5 rows
Lab3.head()

,Entity,alternative name,story,YEAR,records lost,ORGANISATION,METHOD OF LEAK,interesting story,NO OF RECORDS STOLEN,DATA SENSITIVITY,UNUSED,UNUSED.1,Exclude,Unnamed: 13,1st source link,2nd source link,3rd source,source name
0,NaN,NaN,Elaboration if there's an interesting story or...,"years are encoded (0=2004, 8 = 2012, 9 = 2013,...","(use 3m, 4m, 5m or 10m to approximate unknown ...",NaN,NaN,NaN,"(use 3m, 4m, 5m or 10m to approximate unknown ...",1. Just email address/Online information 20 SS...,NaN,NaN,Show this item in the viz?,NaN,NaN,NaN,NaN,NaN
1,AOL,American Online,A former America Online software engineer stol...,0,92000000,web,inside job,NaN,92000000,1,NaN,NaN,NaN,NaN,http://money.cnn.com/2004/06/23/technology/aol...,http://www.msnbc.msn.com/id/8985989/#.UFcN8RgUwaA,NaN,CNN
2,Automatic Data Processing,"Business outsourcing, payrolls, benefits",NaN,1,125000,financial,poor security,NaN,130000,20,NaN,NaN,NaN,NaN,http://abcnews.go.com/Technology/story?id=2160...,NaN,NaN,ABC
3,Ameritrade Inc.,Computer backup tape containing personal infor...,online broker,1,200000,financial,lost / stolen device,NaN,200000,20,NaN,NaN,NaN,NaN,http://www.nbcnews.com/id/7561268/,NaN,NaN,NaN
4,Citigroup,NaN,Blame the messenger! A box of computer tapes c...,1,3900000,financial,lost / stolen device,y,3900000,300,NaN,NaN,NaN,NaN,http://www.nytimes.com/2005/06/07/business/07d...,NaN,NaN,NY Times


In [3]:
#Drop the first row
Lab3 = Lab3.drop(Lab3.index[[0]])

In [4]:
#Drop unnecessary columns from the sheet
drop1 = ['story','alternative name','NO OF RECORDS STOLEN','interesting story','UNUSED' , 'UNUSED.1', 'Exclude', 'Unnamed: 13', '1st source link', '2nd source link', '3rd source', 'source name']
Lab3 = Lab3[[col for col in Lab3.columns if col not in drop1]]

In [5]:
#Rename columns of sheet1 having inconsistencies in name
Lab3 = Lab3.rename(columns = {'Entity':'ENTITY','records lost':'RECORDS LOST' })

In [8]:
Lab3.loc[Lab3['DATA SENSITIVITY'] == 1, 'DATA SENSITIVITY'] = 'Just email address/Online information'
Lab3.loc[Lab3['DATA SENSITIVITY'] == 300, 'DATA SENSITIVITY'] = 'Credit card information'
Lab3.loc[Lab3['DATA SENSITIVITY'] == 4000, 'DATA SENSITIVITY'] = 'Email password/Health records'
Lab3.loc[Lab3['DATA SENSITIVITY'] == 50000, 'DATA SENSITIVITY'] = 'Full bank account details'

#Encountered some string values in column with numeric values
Lab3.loc[Lab3['DATA SENSITIVITY'] == '20', 'DATA SENSITIVITY'] = 'SSN/Personal details'
Lab3.loc[Lab3['DATA SENSITIVITY'] == '300', 'DATA SENSITIVITY'] = 'Credit card information'

In [9]:
#Approximating the UNKNOWN FIGURES

Lab3.loc[Lab3['ENTITY'] == 'OVH', 'RECORDS LOST'] = 5000000
Lab3.loc[Lab3['ENTITY'] == 'UbiSoft', 'RECORDS LOST'] = 10000000

In [10]:
#Change values for column YEAR

Lab3['YEAR'] = Lab3['YEAR'].replace([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14],
[2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2017])

In [11]:
#Replacing {} brackets with the entity name and using the fuzz.partial_ratio to get the entities that match the yahoo Finance atleast 80%

def get_symbol(entity_name):
    url = "http://d.yimg.com/autoc.finance.yahoo.com/autoc?query={}&region=1&lang=en".format(entity_name)

    result = rq.get(url).json()

    for x in result['ResultSet']['Result']:
        if fuzz.partial_ratio(entity_name, x['name']) >= 80:
            if x['exchDisp'] == 'NYSE' or x['exchDisp'] == 'NASDAQ':
                return x['symbol'] 

In [12]:
#Creating a container list to get close dates
ticker = []

In [13]:
Entity_name = Lab3['ENTITY'].tolist()
for y in Entity_name:
    ticker.append(get_symbol(y))    #Once we get the symbol, append it to the empty ticker[].

In [14]:
Stock_price = pd.DataFrame(columns = ['Symbol','Date', 'Close'])
for x in ticker:
    if x is not None:
        try:
            result = pdr.get_data_yahoo(x)
            result = result.reset_index()  
            result['Symbol'] = x
            result = result[['Symbol', 'Date', 'Close']]
            Stock_price = Stock_price.append(result)
        except:
            print('No Ticker')
    else:
        pass

No Ticker
No Ticker
No Ticker
No Ticker
No Ticker
No Ticker


In [15]:
Lab3.insert(loc = 6,column = 'Ticker_Symbol', value = ticker)

In [17]:
Lab3_Updated = pd.merge(Lab3, Stock_price, left_on='Ticker_Symbol',right_on='Symbol', how='right')

In [18]:
Lab3_Updated.to_csv('Lab3_CleanedSheet.csv',index=False)